In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

from dataset import DataSet
import decoder_model

In [2]:
today = datetime.strftime(datetime.now(), '%d%m%y')
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_string('checkpoint',
                    '/home/rayner/tensorflow/models/inception/cnn_depth_tensorflow/300617/train',
                    'coarse model checkpoint')
flags.DEFINE_integer('batch_size', 654, 'Batch size')
flags.DEFINE_string('train_file', 'train.csv', 'Train file')
flags.DEFINE_string('test_file', 'test.csv', 'Test file')

In [3]:
with tf.Graph().as_default():
    global_step = tf.Variable(0, trainable=False)
    dataset = DataSet(FLAGS.batch_size)
    images, depths, invalid_depths = dataset.csv_inputs(FLAGS.test_file,
                                                        target_size=[228, 304])
    keep_conv = tf.placeholder(tf.float32)
    keep_hidden = tf.placeholder(tf.float32)
    logits = decoder_model.model(images, keep_conv, keep_hidden)
    loss = decoder_model.loss(logits, depths, invalid_depths)
#     train_op = op.train(loss, global_step, FLAGS.batch_size)
#     init_op = tf.initialize_all_variables()
    
    sess = tf.Session(config=tf.ConfigProto(
                          log_device_placement=True,
                          gpu_options=tf.GPUOptions(visible_device_list='1')
                      ))
    
#     sess.run(init_op)
    tf.global_variables_initializer()
    
    # parameters
    params = {}
    for variable in tf.trainable_variables():
        variable_name = variable.name
        print("parameter: %s" % (variable_name))
        if variable_name.find("/") < 0 or variable_name.count("/") != 1:
            continue
        params[variable_name] = variable
    # define saver
#     print(params)
    saver = tf.train.Saver(params)
    # load saved model
    ckpt = tf.train.get_checkpoint_state(FLAGS.checkpoint)
    if ckpt and ckpt.model_checkpoint_path:
        print("Loading pretrained model...")
        saver.restore(sess, ckpt.model_checkpoint_path)
        print("Pretrained model restored.")
    else:
        print("No pretrained model.")
        exit(1)

    # train
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    loss_value, logits_val, images_val = sess.run(
            [loss, logits, images],
            feed_dict={keep_conv: 0.8, keep_hidden: 0.5})
    print("test loss %f" % (loss_value))
    
    coord.request_stop()
    coord.join(threads)
#     sess.close()

parameter: coarse1/weights:0
parameter: coarse1/biases:0
parameter: coarse2/weights:0
parameter: coarse2/biases:0
parameter: coarse3/weights:0
parameter: coarse3/biases:0
parameter: coarse4/weights:0
parameter: coarse4/biases:0
parameter: coarse5/weights:0
parameter: coarse5/biases:0
parameter: coarse6/weights:0
parameter: coarse6/biases:0
parameter: coarse7/weights:0
parameter: coarse7/biases:0
parameter: up1/weights:0
parameter: up1/biases:0
parameter: up2/weights:0
parameter: up2/biases:0
parameter: up3/weights:0
parameter: up3/biases:0
parameter: up4/weights:0
parameter: up4/biases:0
parameter: up5/weights:0
parameter: up5/biases:0
parameter: up6/weights:0
parameter: up6/biases:0
Loading pretrained model...
Pretrained model restored.


ResourceExhaustedError: OOM when allocating tensor with shape[654,55,74,256]
	 [[Node: ResizeNearestNeighbor_4 = ResizeNearestNeighbor[T=DT_FLOAT, align_corners=false, _device="/job:localhost/replica:0/task:0/gpu:0"](up4/up4, size_6)]]

Caused by op u'ResizeNearestNeighbor_4', defined at:
  File "/home/rayner/miniconda2/envs/tf/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/rayner/miniconda2/envs/tf/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/rayner/miniconda2/envs/tf/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/rayner/miniconda2/envs/tf/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/rayner/miniconda2/envs/tf/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/ra

In [ ]:
print(ckpt)

In [ ]:
print(logits_val[0,:,:,0].shape)

In [ ]:
for i in range(8):
    plt.figure()
    plt.imshow(logits_val[i,:,:,0])
    plt.axis('off')
    plt.show()

    plt.figure()
    plt.imshow(images_val[i,:,:,0])
    plt.axis('off')
    plt.show()